In [1]:
import tensorflow as tf
from tensorflow import keras


In [2]:
import keras_tuner as kt


In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2021-09-10 13:22:24.350327: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 47220 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:67:00.0, compute capability: 7.5
2021-09-10 13:22:24.351874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46865 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:68:00.0, compute capability: 7.5


In [7]:
#Create a callback to stop training early after reaching a certain value for the validation loss.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 01s]
val_accuracy: 0.8769166469573975

Best val_accuracy So Far: 0.8924999833106995
Total elapsed time: 00h 12m 28s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5093 - accuracy: 0.8209 - val_loss: 0.4304 - val_accuracy: 0.8410
Epoch 2/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3792 - accuracy: 0.8626 - val_loss: 0.3673 - val_accuracy: 0.8648
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3397 - accuracy: 0.8734 - val_loss: 0.3474 - val_accuracy: 0.8722
Epoch 4/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3096 - accuracy: 0.8855 - val_loss: 0.3301 - val_accuracy: 0.8814
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2948 - accuracy: 0.8911 - val_loss: 0.3329 - val_accuracy: 0.8816
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2761 - accuracy: 0.8983 - val_loss: 0.3358 - val_accuracy: 0.8807
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2638 - accuracy: 0.9023 - val_loss: 0.3362 - val_accuracy:

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)